# CbO

In [6]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [7]:
G = {1, 2, 3, 4, 5}
M = {'a', 'b', 'c', 'd'}
M_G = {
    'a': {2, 4, 5},
    'b': {1, 2, 3, 5},
    'c': {1, 2, 3},
    'd': {1, 4, 5},
}
G_M = {
    1: {'b', 'c', 'd'},
    2: {'a', 'b', 'c'},
    3: {'b', 'c'},
    4: {'a', 'd'},
    5: {'a', 'b', 'd'}
}

In [8]:
# simple functions for getting derivatives from sets (for both "dimensions")

def derivative_g(g):
    if len(g) == 0:
        return M
    
    der = M.copy()
    for i in g:
        der &= G_M[i]

    return der

def derivative_m(m):
    if len(m) == 0:
        return G
    
    der = G.copy()
    for i in m:
        der &= M_G[i]

    return der

# Equivalence classes

In [9]:
# algorithm implementation for computing the set of equivalence classes

powerset_of_M = list(sorted(set(x)) for x in powerset(M))
eq_classes = {}

for s in powerset_of_M:
    s_d = frozenset(derivative_g(derivative_m(s)))
    if s_d in eq_classes:
        eq_classes[s_d].append(s)
    else:
        eq_classes[s_d] = [s]

for [key, value] in eq_classes.items():
    print(derivative_m(key), value)

{1, 2, 3, 4, 5} [[]]
{1, 2, 3} [['c'], ['b', 'c']]
{2, 4, 5} [['a']]
{1, 2, 3, 5} [['b']]
{1, 4, 5} [['d']]
{2} [['a', 'c'], ['a', 'b', 'c']]
{1} [['c', 'd'], ['b', 'c', 'd']]
{2, 5} [['a', 'b']]
{4, 5} [['a', 'd']]
{1, 5} [['b', 'd']]
set() [['a', 'c', 'd'], ['a', 'b', 'c', 'd']]
{5} [['a', 'b', 'd']]


# Minimal generators

In [10]:
# algorithm implementation for computing the set of minimal generators

def is_min_generator(eq_class):
    eq_class_closure = derivative_g(derivative_m(eq_class))
    pset = list(sorted(set(x)) for x in powerset(elem))
    for i in pset:
        if (i == eq_class):
            continue
        if (derivative_g(derivative_m(i)) == eq_class_closure):
            return False
    return True

min_generators = {}

for [key, value] in eq_classes.items():
    for elem in value:
        if (not is_min_generator(elem)):
            continue

        if key in min_generators:
            min_generators[key].append(elem)
        else:
            min_generators[key] = [elem]
        

for [key, value] in min_generators.items():
    print(derivative_m(key), value)

{1, 2, 3, 4, 5} [[]]
{1, 2, 3} [['c']]
{2, 4, 5} [['a']]
{1, 2, 3, 5} [['b']]
{1, 4, 5} [['d']]
{2} [['a', 'c']]
{1} [['c', 'd']]
{2, 5} [['a', 'b']]
{4, 5} [['a', 'd']]
{1, 5} [['b', 'd']]
set() [['a', 'c', 'd']]
{5} [['a', 'b', 'd']]


# Generator implication cover

In [11]:
nmingen = []
for key in eq_classes:
    if len(eq_classes[key]) == 1:
        continue
    nmingen += min_generators[key]

for f in nmingen:
    print(f'{f} => {derivative_g(derivative_m(f)).difference(f)}')

['c'] => {'b'}
['a', 'c'] => {'b'}
['c', 'd'] => {'b'}
['a', 'c', 'd'] => {'b'}


In [12]:
proper_premises = {}

for s in powerset_of_M:
    s_d = frozenset(derivative_g(derivative_m(s)))
    s_union = set(s)
    for el in s:
        s_union |= derivative_g(derivative_m(set(s) - set(el)))
                                
    if s_d == s_union:
        continue
    proper_premises[s_d] = set(s)

for [key, value] in proper_premises.items():
    print(f'{value} => {set(key) - value}')

{'c'} => {'b'}


# Find Duquenne-Guigues bases

In [13]:
G = {12, 15, 23, 24, 25, 34, 35}
M = {'a', 'b', 'c', 'd', 'e'}
M_G = {
    'a': {35},
    'b': {15, 23},
    'c': {15, 23, 24, 34},
    'd': {12, 15, 25},
    'e': {12, 35}
}
G_M = {
    12: {'d', 'e'},
    15: {'b', 'c', 'd'},
    23: {'b', 'c'},
    24: {'c'},
    25: {'d'},
    34: {'c'},
    35: {'a', 'e'},
}

def is_pseudointent(p):
    if derivative_g(derivative_m(p)) == p:
        return False

    power_p = list(set(x) for x in powerset(p))
    for q in power_p[:-1]:
        q_dd = derivative_g(derivative_m(q))

        if len(q) ==0:
            continue
        if is_pseudointent(q) and not (q_dd.issubset(p) and len(p) != len(q_dd)):
            return False
    return True

power_M = list(set(x) for x in powerset(M))
im_base_classes = []
for p in power_M:
    if not is_pseudointent(p):
        continue
    print(p, '=>', derivative_g(derivative_m(p)).difference(p))

{'a'} => {'e'}
{'b'} => {'c'}
{'c', 'e'} => {'a', 'b', 'd'}
{'c', 'd'} => {'b'}
{'a', 'e', 'd'} => {'c', 'b'}
